# Configurando e testando prompts

In [1]:
#@title Instalar dependências e carregar pacotes do git
# carregando o get_git para importar utilitários do github
import os
if not os.path.isfile('./get_git.py'):
   !curl https://raw.githubusercontent.com/luizanisio/llms/refs/heads/main/util/get_git.py -o ./get_git.py
import get_git
get_git.deps() # instala unsloth, Transformers, Rouge e Levenshtein se precisar
Util = get_git.sync_git_util()
Util.dados_python()
Prompt = get_git.sync_git_prompt()
UtilAgentes = get_git.sync_git_agentes()

✅ Instalação já realizada nesse ambiente _o/
[OK] Dependências de análise instaladas _o/
✅ Instalação já realizada nesse ambiente _o/
[OK] Dependências de treinamento instaladas _o/
[OK] Util carregado de /content/src (cache)
Util carregado corretamente em 2025-08-09 14:01:02!
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
[OK] Prompt carregado de /content/src (cache)
Transformers: 4.53.3 2025.7.1
Tourch: 2.7.0+cu126
[OK] UtilAgentes carregado de /content/src (cache)
UtilAgentes carregado corretamente em 2025-08-09 14:01:22!


In [2]:
#@title Lista os modelos disponíveis
Prompt.listar_modelos()

Modelos disponíveis:

- MODELO_GEMMA3_1B: google/gemma-3-1b-it
- MODELO_GEMMA3_4B: google/gemma-3-4b-it
- MODELO_GEMMA3_12B: google/gemma-3-12b-it
- MODELO_GEMMA3_27B: google/gemma-3-27b-it
- MODELO_QWEN_7B: Qwen/Qwen2.5-7B-Instruct-1M
- MODELO_JUREMA_7B: Jurema-br/Jurema-7B
- MODELO_QWEN_14B: Qwen/Qwen2.5-14B-Instruct-1M

* Utilize: Prompt.modelos.MODELO....


In [ ]:
#@title Escolhe e carrega o modelo
p = Prompt(modelo=Prompt.modelos.MODELO_GEMMA3_4B, max_seq_length=4096)

In [4]:
#@title Pergunta ao modelo
res = p.prompt('Complete os próximos dois números da sequência e explique de forma objetiva em uma frase: 1, 1, 2, 3, 5, 8 ...')
print('RESPOSTA:', res)

RESPOSTA: Os próximos dois números da sequência são 13 e 21.

Em uma frase objetiva: Esta sequência representa a sequência de Fibonacci, onde cada número é a soma dos dois números anteriores.


In [5]:
#@title Resposta em json do modelo
import json
pergunta = '''Retorne um json válido com a estrutrua {"mensagem": com a mensagem do usuário, "itens": com uma lista de itens quando ele enumerar algo }
              Mensagem do usuário: Eu preciso comprar abacaxi, pera e 2L de leite.'''
r = p.prompt_to_json(pergunta)
print(json.dumps(r, indent=2, ensure_ascii=False))

{
  "mensagem": "Eu preciso comprar abacaxi, pera e 2L de leite.",
  "itens": [
    "abacaxi",
    "pera",
    "2L de leite"
  ],
  "usage": {
    "input_tokens": 64,
    "output_tokens": 60,
    "time": 4.680930852890015
  }
}


# Configurando agentes

In [14]:
#@title Tarefa básica de exemplo
from IPython.display import clear_output
print('Carregando modelo ...')
p = Prompt(modelo=Prompt.modelos.MODELO_GEMMA3_4B, max_seq_length=4096)
clear_output()
print(f'Modelo {p.modelo} carregado _o/')
print('Preparando agentes ...')
servicos = UtilAgentes.AgentesToolsBasicos.get_servicos_basicos()
print('Definindo tarefa ...')

tarefa = UtilAgentes.Tarefa(nome='Tarefa 001',
                descricao='Descobrir que dia é hoje e encontrar informações sobre o que é Xibunfa e como podemos usar Xibunfa para limpar o chão.',
                objetivo='Descrever o que é Xibunfa e explicar como Xibunfa pode ser usada na limpeza do chão.',
                servicos_disponiveis=servicos,
                call_llm = p.prompt_to_json)

rt = UtilAgentes.ResolverTarefas([tarefa])
print('Resolvendo tarefa ...\n---------------------')
rt.resolver()

Modelo Modelos.MODELO_GEMMA3_4B carregado _o/
Preparando agentes ...
Definindo tarefa ...
Resolvendo tarefa ...
---------------------
14:12:20|RESOLVENDO TAREFA - : Tarefa 001 com 2 serviços disponíveis
14:12:20|DESCOBERTA - : Descobrindo serviços para a tarefa Tarefa 001
14:12:30|DESCOBERTA CONCLUÍDA - : Serviços descobertos para a tarefa Tarefa 001: 2
14:12:30|SERVIÇO SUGERIDO - : DataHora - O usuário precisa saber que dia é hoje.
14:12:30|SERVIÇO SUGERIDO - : Busca - O usuário precisa de informações sobre o que é Xibunfa e como usá-lo para limpar o chão.
14:12:30|ACIONANDO SERVIÇOS - : DataHora,Busca
14:12:30|PREPARANDO SERVIÇO - : DataHora: {}
14:12:30|SERVIÇO ACIONADO - : DataHora: {}
14:12:30|CONHECIMENTO - : Calendário com dia, mês e ano | pg. None | score 0
14:12:30|PREPARANDO SERVIÇO - : Busca: {"palavras": ["Xibunfa", "limpeza chão"]}
14:12:30|SERVIÇO ACIONADO - : Busca: {"palavras": ["Xibunfa", "limpeza chão"]}
ACIONANDO AGENTE DE TAREFA: Padrão
AVALIANDO RESPOSTA LLM:
{
  "

In [15]:
#@title Imprimindo solução da tarefa
tarefa.print(True)

TAREFA: Tarefa 001
DESCRIÇÃO: Descobrir que dia é hoje e encontrar informações sobre o que é Xibunfa e como podemos usar Xibunfa para limpar o chão.
OBJETIVO: Descrever o que é Xibunfa e explicar como Xibunfa pode ser usada na limpeza do chão.
---------------------------------------
CONHECIMENTO UTILIZADO:
	 - Calendário com dia, mês e ano (pg: None | sc: 0): Hoje é o dia 09 de August de 2025
---------------------------------------
SOLUÇÃO INCOMPLETA: Hoje é 09 de agosto de 2025. A Base de Conhecimento informa que Xibunfa é um produto para limpeza de chão. No entanto, a informação sobre como utilizar Xibunfa para limpar o chão não está disponível.
